In [1]:
import pandas as pd
from pulp import LpMinimize, LpMaximize, LpProblem, LpVariable

In [2]:
cnap = pd.read_csv("core_nutrient_amounts_prices_v3.csv", index_col=0)
cnap = cnap.rename(columns={"name": "nutrient_name"})
constraints = (
    pd.read_csv("nutrient_constraints_csv.csv")
    .set_index("nutrient_nbr")
    .drop(columns=["id", "rank"])
    .rename(columns={"name": "nutrient_name"})
    .iloc[:, :5]
)
constraints = constraints.loc[
    constraints.target.notna() | constraints.ll.notna() | constraints.ul.notna()
]
n_values = cnap.pivot(
    index="nutrient_nbr", columns="food", values="nutrient_unit_per_dollar"
)
cc = constraints.join(n_values, how="left")
first_food_idx = cc.columns.get_loc("Almonds")
cc.iloc[:, first_food_idx:] = cc.iloc[:, first_food_idx:].fillna(0)

In [3]:
prob = LpProblem("Minimize_Cost", LpMinimize)
decision_variables = []
nc = []
c_num = 1
first_food_idx = cc.columns.get_loc("Almonds")
for food in cc.columns[first_food_idx:]:
    decision_variables.append(LpVariable(f"{food}", lowBound=0))
prob += sum([decision_variable for decision_variable in decision_variables])
for i, row in cc.iterrows():
    constraint = 0
    for j, food in enumerate(cc.columns[first_food_idx:]):
        constraint += decision_variables[j] * row[food]
    if pd.notna(row.target):
        nc.append((row.nutrient_name, "target", c_num))
        print(row.nutrient_name, row.target)
        prob += constraint == row.target
        c_num += 1
    elif pd.notna(row.ll):
        nc.append((row.nutrient_name, "ll", c_num))
        prob += constraint >= row.ll
        c_num += 1
    elif pd.notna(row.ul):
        nc.append((row.nutrient_name, "ul", c_num))
        prob += constraint <= row.ul
        c_num += 1

Energy 2500.0


In [4]:
prob.solve()
for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)

Beef_Liver = 0.063653391
Black_Beans = 0.094690031
Broccoli = 0.078200251
Cabbage = 0.091966557
Carrots = 0.064205073
Chickpeas = 0.024193204
Eggs = 0.11585754
Salmon = 1.483472
Sunflower_seeds = 0.045311149
Tomatoes = 0.485888
Wheat_flour = 0.4163895
White_Beans = 0.24165364
Whole_Milk = 0.42423894


In [5]:
fs = pd.DataFrame(
    [(v.name, v.varValue) for v in prob.variables() if v.varValue != 0],
    columns=["food", "spend"],
).iloc[:-1, :]
cn = cc.set_index("nutrient_name").iloc[:, 4:]
# Rank the foods for each nutrient
food_ranks = cn.rank(axis=1, method="min", ascending=False)
final_foods = cnap.loc[
    cnap.food.isin([v.name for v in prob.variables() if v.varValue != 0])
].drop_duplicates("food")

In [6]:
ffs = final_foods.merge(fs, on="food")

In [7]:
ffs.spend / ffs.price_per_100_g

0    2.843805
1    0.375779
2    0.556204
3    0.206227
4    0.841114
5    0.350347
6    0.075708
dtype: float64

In [8]:
ffs.spend.sum() * 365

855.4806581249999

In [9]:
14 * 100 / 453.592

3.086474188257289

In [10]:
food_ranks["Sardines"].sort_values()

nutrient_name
PUFA 20:5 n-3 (EPA)                1.0
Caffeine                           1.0
Vitamin B-12                       2.0
PUFA 22:6 n-3 (DHA)                2.0
Cholesterol                        4.0
Calcium, Ca                        5.0
Lycopene                           6.0
Vitamin D (D2 + D3)                7.0
Retinol                            8.0
PUFA 18:3                         11.0
Selenium, Se                      12.0
PUFA 22:5 n-3 (DPA)               15.0
Niacin                            21.0
Iron, Fe                          23.0
Phosphorus, P                     23.0
PUFA 18:2                         24.0
Sodium, Na                        24.0
Carotene, alpha                   25.0
Vitamin A, RAE                    26.0
Choline, total                    26.0
Protein                           26.0
Total lipid (fat)                 27.0
Vitamin E (alpha-tocopherol)      27.0
Fatty acids, total saturated      33.0
Riboflavin                        37.0
Zinc, Zn   

In [11]:
food_ranks["Salmon"].sort_values()

nutrient_name
PUFA 22:5 n-3 (DPA)                1.0
Caffeine                           1.0
PUFA 20:5 n-3 (EPA)                2.0
PUFA 22:6 n-3 (DHA)                3.0
Lycopene                           6.0
Vitamin D (D2 + D3)                6.0
Retinol                            9.0
Vitamin B-12                      11.0
Cholesterol                       16.0
Carotene, alpha                   25.0
Vitamin A, RAE                    27.0
Niacin                            28.0
Vitamin E (alpha-tocopherol)      29.0
Fatty acids, total saturated      32.0
Selenium, Se                      32.0
PUFA 18:3                         33.0
PUFA 18:2                         34.0
Sodium, Na                        35.0
Total lipid (fat)                 35.0
Choline, total                    41.0
Protein                           42.0
Vitamin B-6                       45.0
Vitamin C, total ascorbic acid    52.0
Phosphorus, P                     53.0
Carotene, beta                    56.0
Vitamin K (

In [12]:
# Create a dataframe to store the nutritional content of the optimal solution
nutritional_content = pd.DataFrame(
    index=cc.index, columns=["nutrient_name", "unit_name", "total_amount"]
)

# Fill in the nutrient names and unit names
nutritional_content["nutrient_name"] = cc["nutrient_name"]
nutritional_content["unit_name"] = cc["unit_name"]

# Calculate the total amount of each nutrient in the optimal solution
for i, row in cc.iterrows():
    total_amount = 0
    for j, food in enumerate(cc.columns[first_food_idx:]):
        total_amount += decision_variables[j].varValue * row[food]
    nutritional_content.at[i, "total_amount"] = total_amount

nutritional_content

,nutrient_name,unit_name,total_amount
nutrient_nbr,,,
203.0,Protein,G,119.970437
204.0,Total lipid (fat),G,64.739769
205.0,"Carbohydrate, by difference",G,384.997455
208.0,Energy,KCAL,2499.999991
262.0,Caffeine,MG,0.0
291.0,"Fiber, total dietary",G,66.458351
301.0,"Calcium, Ca",MG,1000.000001
303.0,"Iron, Fe",MG,25.064753
304.0,"Magnesium, Mg",MG,830.558329


In [13]:
content_constraints = nutritional_content.join(constraints[["target", "ll", "ul"]])

In [14]:
content_constraints.head()

,nutrient_name,unit_name,total_amount,target,ll,ul
nutrient_nbr,,,,,,
203.0,Protein,G,119.970437,NaN,62.500000,218.750000
204.0,Total lipid (fat),G,64.739769,NaN,55.555556,97.222222
205.0,"Carbohydrate, by difference",G,384.997455,NaN,281.250000,406.250000
208.0,Energy,KCAL,2499.999991,2500.0,NaN,NaN
262.0,Caffeine,MG,0.0,NaN,NaN,400.000000


In [15]:
delta = 0.01
content_constraints["ul_tight"] = (
    abs(
        (content_constraints["total_amount"] - content_constraints["ul"])
        / content_constraints["ul"]
    )
    < delta
)
content_constraints["ll_tight"] = (
    abs(
        (content_constraints["total_amount"] - content_constraints["ll"])
        / content_constraints["ll"]
    )
    < delta
)
content_constraints["target_tight"] = (
    abs(
        (content_constraints["total_amount"] - content_constraints["target"])
        / content_constraints["target"]
    )
    < delta
)

In [16]:
content_constraints.loc[
    content_constraints["ul_tight"]
    | content_constraints["ll_tight"]
    | content_constraints["target_tight"]
]

,nutrient_name,unit_name,total_amount,target,ll,ul,ul_tight,ll_tight,target_tight
nutrient_nbr,,,,,,,,,
208.0,Energy,KCAL,2499.999991,2500.0,NaN,NaN,False,False,True
301.0,"Calcium, Ca",MG,1000.000001,NaN,1000.00,NaN,False,True,False
306.0,"Potassium, K",MG,4699.999992,NaN,4700.00,NaN,False,True,False
322.0,"Carotene, alpha",UG,1500.000001,NaN,1500.00,NaN,False,True,False
323.0,Vitamin E (alpha-tocopherol),MG,15.0,NaN,15.00,NaN,False,True,False
328.0,Vitamin D (D2 + D3),UG,15.0,NaN,15.00,NaN,False,True,False
337.0,Lycopene,UG,8000.00005,NaN,8000.00,21000.0,False,True,False
401.0,"Vitamin C, total ascorbic acid",MG,90.0,NaN,90.00,NaN,False,True,False
421.0,"Choline, total",MG,549.999996,NaN,550.00,NaN,False,True,False


In [17]:
for name, constraint in prob.constraints.items():
    print(f"Shadow price of {name}: {constraint.pi}")

Shadow price of _C1: 0.0
Shadow price of _C2: 0.0
Shadow price of _C3: 0.0
Shadow price of _C4: 0.00012699354
Shadow price of _C5: 0.0
Shadow price of _C6: 0.0
Shadow price of _C7: 0.00025742448
Shadow price of _C8: 0.0
Shadow price of _C9: 0.0
Shadow price of _C10: 0.0
Shadow price of _C11: 3.4370204e-05
Shadow price of _C12: 0.0
Shadow price of _C13: 0.0
Shadow price of _C14: 0.0
Shadow price of _C15: 0.0
Shadow price of _C16: 0.0
Shadow price of _C17: 0.0
Shadow price of _C18: 0.0
Shadow price of _C19: 3.0640925e-05
Shadow price of _C20: 0.015024443
Shadow price of _C21: 0.014186977
Shadow price of _C22: 2.8993568e-05
Shadow price of _C23: 0.0036329362
Shadow price of _C24: 0.0
Shadow price of _C25: 0.0
Shadow price of _C26: 0.0
Shadow price of _C27: 0.0
Shadow price of _C28: 0.0
Shadow price of _C29: 0.00091438525
Shadow price of _C30: 0.0
Shadow price of _C31: 0.0
Shadow price of _C32: -6.5202923e-06
Shadow price of _C33: 0.0
Shadow price of _C34: 0.0010513966
Shadow price of _C35

In [18]:
nc

[('Protein', 'll', 1),
 ('Total lipid (fat)', 'll', 2),
 ('Carbohydrate, by difference', 'll', 3),
 ('Energy', 'target', 4),
 ('Caffeine', 'ul', 5),
 ('Fiber, total dietary', 'll', 6),
 ('Calcium, Ca', 'll', 7),
 ('Iron, Fe', 'll', 8),
 ('Magnesium, Mg', 'll', 9),
 ('Phosphorus, P', 'll', 10),
 ('Potassium, K', 'll', 11),
 ('Sodium, Na', 'ul', 12),
 ('Zinc, Zn', 'll', 13),
 ('Copper, Cu', 'll', 14),
 ('Selenium, Se', 'll', 15),
 ('Retinol', 'll', 16),
 ('Vitamin A, RAE', 'll', 17),
 ('Carotene, beta', 'll', 18),
 ('Carotene, alpha', 'll', 19),
 ('Vitamin E (alpha-tocopherol)', 'll', 20),
 ('Vitamin D (D2 + D3)', 'll', 21),
 ('Lycopene', 'll', 22),
 ('Vitamin C, total ascorbic acid', 'll', 23),
 ('Thiamin', 'll', 24),
 ('Riboflavin', 'll', 25),
 ('Niacin', 'll', 26),
 ('Vitamin B-6', 'll', 27),
 ('Vitamin B-12', 'll', 28),
 ('Choline, total', 'll', 29),
 ('Vitamin K (phylloquinone)', 'll', 30),
 ('Folate, DFE', 'll', 31),
 ('Cholesterol', 'ul', 32),
 ('Fatty acids, total saturated', 'ul

In [35]:
pis = []

for name, constraint in prob.constraints.items():
    pis.append(constraint.pi)

In [41]:
constraint_shadow_prices = pd.DataFrame(
    nc, columns=["nutrient_name", "constraint_type", "constraint_num"]
).assign(shadow_price=pis)

In [43]:
constraint_shadow_prices.sort_values(by="shadow_price", ascending=False)

,nutrient_name,constraint_type,constraint_num,shadow_price
37,PUFA 22:5 n-3 (DPA),ll,38,5.046565
34,PUFA 18:3,ll,35,0.044254
19,Vitamin E (alpha-tocopherol),ll,20,0.015024
20,Vitamin D (D2 + D3),ll,21,0.014187
22,"Vitamin C, total ascorbic acid",ll,23,0.003633
33,PUFA 18:2,ll,34,0.001051
28,"Choline, total",ll,29,0.000914
6,"Calcium, Ca",ll,7,0.000257
3,Energy,target,4,0.000127
10,"Potassium, K",ll,11,0.000034


# TODO

1. Create robust pipelines to acquire and then combine data from both legacy and fndds sources.
2. Ensure there exists robust optimization process that produces nice, interpretable outputs.
3. Determine what the vars on each constraint mean.
